In [0]:
from pyspark.sql import functions as f
from functools import reduce

In [0]:
rekurze = spark.read.table("rakathon_hackathon.rekurze.rekurze")

In [0]:
display(rekurze.groupBy("id").agg(f.count("*").alias("id_count"))
        .orderBy(f.desc("id_count"))
        )

print(rekurze.count())
print(rekurze.select("id").distinct().count())
# We have 1 row per 1 id

In [0]:
display(rekurze)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
display(rekurze.select("id","rok_dg","mesic_dg","grading","pl_delka","nl_delka","novotvar_poradi","vekova_kategorie_10let_dg","umrti","nl_typ_lecby_1","nl_typ_lecby_2","pl_typ_lecby_1","pl_typ_lecby_2","pl_typ_lecby_3")
        .filter(f.col("pl_delka").isNull())
        #.filter(f.col("nl_delka").isNull())
        #.count()
        )

In [0]:
display(rekurze.filter(f.col("time_datum_dg_to_zahajeni_nl").isNull()).count())
display(rekurze.filter(f.col("nl_delka").isNull()).count())

In [0]:
display(rekurze
        .filter(f.col("novotvar_poradi")==1)
        .filter(f.col("nl_delka").isNotNull())
        .count()
        )

In [0]:
display(rekurze
        #.filter(f.col("novotvar_poradi")==1)
        .filter(f.col("nl_delka").isNotNull())
        #.count()
        )

Databricks visualization. Run in Databricks to view.

In [0]:
display(rekurze.drop("stadium").count())
display(rekurze.drop("stadium").dropDuplicates().count())

In [0]:
high_stadium = [1,2]

display(rekurze
        .filter(f.col("stadium").isin(high_stadium))
        #.filter(f.col("nl_delka").isNotNull())
        #.select("stadium","nl_delka","novotvar_poradi","umrti")
        .count()
        )

In [0]:
display(rekurze.select("id","stadium","nl_delka","novotvar_poradi","umrti"))
# 1-2 stadium se muze jednat o relaps

In [0]:
display(rekurze.select("tnm_klasifikace_t_kod","tnm_klasifikace_n_kod","tnm_klasifikace_m_kod"))

# t kod -> musi obsahovat "4"
# NEBO
# n kod -> musi obsahovat "3"
# NEBO
# m kod -> musi obsahovat "1"

# Tady nemá cenu řešit, vyřadíme z datasetu

Databricks visualization. Run in Databricks to view.

## cleansed layer

In [0]:
disp_cols = ["je_disp", "je_disp_prakt", "je_disp_onk", "je_disp_int", "je_disp_chir", "je_disp_gyn"]
advanced_stadium = [3,4]

rekurze_recidiva_cleansed = (rekurze#.select("id","stadium","pl_delka","je_disp","je_disp_prakt","je_disp_onk","je_disp_int","je_disp_chir","je_disp_gyn","nl_delka","vekova_kategorie_10let_dg","umrti")
        .withColumn("advanced_stadium",(    
            f.when(f.col("tnm_klasifikace_t_kod").contains("4") |
            f.col("tnm_klasifikace_n_kod").contains("3") |
            f.col("tnm_klasifikace_m_kod").contains("1"),1).otherwise(0))
        )
        .withColumn("disp_bool", f.when(f.greatest(*[f.col(c) for c in disp_cols]) == 1, 1).otherwise(0))
        .withColumn("nl_bool", f.when(f.col("nl_delka").isNotNull(), 1).otherwise(0))
        .withColumn("pl_bool", f.when(f.col("pl_delka").isNotNull(), 1).otherwise(0))
        .withColumn("recidiva", f.when((f.col("advanced_stadium") == 0) & 
                                       (f.col("disp_bool") == 1) & (f.col("nl_bool") == 1) & (f.col("pl_bool") == 1), 1).otherwise(0))
        .withColumn("remise",f.when((f.col("pl_bool") == 1) & (f.col("disp_bool") == 1), 1).otherwise(0))
        .withColumn("target", f.when(f.col("recidiva") == 1, 1).when((f.col("advanced_stadium") == 0) & (f.col("disp_bool") == 1), 0)
            .otherwise(None))
        )

# 24.9 %
print(rekurze_recidiva_cleansed.count()) #95 784
print(rekurze_recidiva_cleansed.filter(f.col("nl_bool")==1).count()) # 57 866
print(rekurze_recidiva_cleansed.filter(f.col("recidiva")==1).count()) # 

In [0]:
display(rekurze_recidiva_cleansed.filter(f.col("nl_pocet_leceb")>0).count()) # 57 866
# - když byla nasledna lecba jenom hormonalni
display(rekurze_recidiva_cleansed.filter(f.col("nl_pocet_leceb")>0)
        .select("id","nl_pocet_leceb","je_nl_oper","je_nl_radio","je_nl_chemo","je_nl_imuno","je_nl_target","je_nl_hormo")) 
display(rekurze_recidiva_cleansed.filter(f.col("nl_pocet_leceb")>0)
        .filter(f.col("je_nl_hormo")==0)
        .count()) # 16 289

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
display(rekurze_recidiva_cleansed.select("id","time_datum_dg_to_zahajeni_pl","time_datum_dg_to_zahajeni_nl","time_datum_dg_to_umrti","pl_delka","nl_delka","disp_bool","recidiva")
        )
print(rekurze_recidiva_cleansed.count())
print(rekurze_recidiva_cleansed.filter(f.col("advanced_stadium")==0).count())

# Po jaké době došlo k recidivě?
# disp_bool = 1 došlo k remisi
#  


In [0]:
from functools import reduce

all_treatments = [
    "je_nl_oper", "je_nl_radio", "je_nl_chemo",
    "je_nl_imuno", "je_nl_target", "je_nl_hormo"
]

nl_other_cols = [
    "je_nl_oper", "je_nl_radio", "je_nl_chemo",
    "je_nl_imuno", "je_nl_target"
]

# All treatment columns except hormo
non_hormo_cols = [c for c in all_treatments if c != "je_nl_hormo"]

# Build condition: hormo == 1 AND all others == 0
only_hormo_condition = (
    (f.col("je_nl_hormo") == 1) &
    reduce(lambda a, b: a & (f.col(b) == 0), non_hormo_cols, f.lit(True))
)

# Add the column
rekurze_recidiva_cleansed_nl = (rekurze_recidiva_cleansed
       # .select("id","nl_pocet_leceb","je_nl_oper","je_nl_radio","je_nl_chemo","je_nl_imuno","je_nl_target","je_nl_hormo")
        .withColumn("nl_only_hormo", f.when(only_hormo_condition, 1).otherwise(0))
        .withColumn("nl_others", f.when(f.greatest(*[f.col(c) for c in nl_other_cols]) == 1, 1).otherwise(0))
        .withColumn("recidiva_new", f.when(f.col("nl_others") == 1, 1).otherwise(0))
        .withColumn("observed_time_window", f.when(f.col("rok_dg") < 2018, 1).otherwise(0))
        .withColumn(
            "target_new",
            f.when((f.col("recidiva_new") == 1) & (f.col("observed_time_window") == 1), 1)
             .when((f.col("advanced_stadium") == 0) & (f.col("observed_time_window") == 1), 0)
             .otherwise(None)
        ))

In [0]:
display(rekurze_recidiva_cleansed_nl.select(f.col("rok_dg")))

Databricks visualization. Run in Databricks to view.

In [0]:
display(rekurze_recidiva_cleansed_nl.select(f.col(target_new)))

In [0]:
#rekurze_recidiva_cleansed_nl.write.mode("overwrite").saveAsTable("rakathon_hackathon.rekurze.rekurze_initial_set")

In [0]:
display(rekurze_recidiva_cleansed_nl
        .select("id","nl_pocet_leceb","je_nl_oper","je_nl_radio","je_nl_chemo","je_nl_imuno","je_nl_target","je_nl_hormo","nl_others","recidiva_new","target_new","observed_time_window")
        

)

Databricks visualization. Run in Databricks to view.

In [0]:
rekurze_recidiva_cleansed_nl_death = (rekurze_recidiva_cleansed_nl
        .filter(f.col("vnejsi_pricina_umrti_kod").isNull())
        .withColumn("cause_of_death_BC",f.when(f.col("prvotni_pricina_umrti_kod").rlike("C5[0-9]{2}"), 1).otherwise(0))
        .withColumn("dead_5_years_bc",f.when((f.col("time_datum_dg_to_umrti") < 5 * 365) & (f.col("cause_of_death_BC") == 1),1).when(
                (f.col("observed_time_window") == 1) & (f.col("cause_of_death_BC") == 0), 0).otherwise(None)
)
        )

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
#rekurze_recidiva_cleansed_nl_death.write.mode("overwrite").saveAsTable("rakathon_hackathon.rekurze.rekurze_death_initial_set")